In [1]:
!pip install owlready2

You should consider upgrading via the '/Users/jorge/PycharmProjects/pizzaOwl/venv/bin/python -m pip install --upgrade pip' command.


Primero importamos el paquete owlready y creamos una ontologia sobre la que trabajaremos

In [2]:
from owlready2 import *

onto = get_ontology("pizza.owl")

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


A continuacion, creamos tres clases, una para representar la pizza y dos para la base y los ingredientes. Estas tres clases son disjuntas, ya que por ejemplo, un ingrediente solo puede pertenecer a la clase PizzaTopping, y no a Pizza o PizzaBase.

In [3]:
with onto:
    class Pizza(Thing): pass
    class PizzaBase(Thing): pass
    class PizzaTopping(Thing): pass
    AllDisjoint([Pizza, PizzaTopping, PizzaBase])

Tanto las clases PizzaBase como PizzaTopping incluyen distintos tipos de base e ingredientes que usaremos para crear pizzas.
Es importante tambien que marquemos estas subclases como disjuntas.

In [4]:
with onto:
    class ThinAndCrispy(PizzaBase): pass
    class DeepPan(PizzaBase): pass
    AllDisjoint([ThinAndCrispy, DeepPan])

    class Meat(PizzaTopping): pass
    class SeaFood(PizzaTopping): pass
    class Cheese(PizzaTopping): pass
    class Vegetable(PizzaTopping): pass
    AllDisjoint([Meat, SeaFood, Cheese, Vegetable])

Del mismo modo, para cada clase ingrediente existen diferentes tipos:

In [5]:
with onto:
    # Meat subclasses
    class Ham(Meat): pass
    class Pepperoni(Meat): pass
    class SpicyBeef(Meat): pass
    class Salami(Meat): pass
    AllDisjoint([Ham, Pepperoni, SpicyBeef, Salami])

    # SeaFood subclasses
    class Anchovy(SeaFood): pass
    class Tuna(SeaFood): pass
    class Prawn(SeaFood): pass
    AllDisjoint([Anchovy, Tuna, Prawn])

    # Cheese subclasses
    class Mozarella(Cheese): pass
    class Parmezan(Cheese): pass
    AllDisjoint([Mozarella, Parmezan])

    # Vegetable subclasses
    class Caper(Vegetable): pass
    class Mushroom(Vegetable): pass
    class Olive(Vegetable): pass
    class Onion(Vegetable): pass
    class Pepper(Vegetable): pass
    class Tomato(Vegetable): pass
    AllDisjoint([Caper, Mushroom, Olive, Onion, Pepper, Tomato])

Por ultimo, tenemos que para la clase Pepper existen diferentes tipos de pimiento:

In [6]:
with onto:
    class RedPepper(Pepper): pass
    class GreenPepper(Pepper): pass
    class JalapenoPepper(Pepper): pass
    AllDisjoint([RedPepper, GreenPepper, JalapenoPepper])

Con todas las clases y subclases que necesitamos ya creadas, el siguiente paso sera definir algunas propiedades que nos serviran para establecer relaciones entre las clases anteriores.

In [7]:
with onto:
    class hasIngredient(ObjectProperty, TransitiveProperty): pass

    class hasTopping(hasIngredient):
        domain = [Pizza]
        range = [PizzaTopping]

    class hasBase(hasIngredient, FunctionalProperty):
        domain = [Pizza]
        range = [PizzaBase]

En el caso de la propiedad hasIngredient es facil ver que es transitiva, ya que si tenemos un ingrediente A que contiene un ingrediente B y a su vez B contiene otro ingrediente C, C tambien es un ingrediente de A.

En cuanto a la propiedad hasTopping, la definimos como una subpropiedad de hasIngredient, con el dominio Pizza y el rango PizzaTopping.

hasBase esta tambien definida a partir de hasIngredient, pero en este caso se trata de una propiedad funcional ya que para cada Pizza solo puede haber una PizzaBase correspondiente.

De manera semejante a como hemos definido las propiedades anteriores, podemos definir sus correspondientes inversas:

In [8]:
with onto:
    class isIngredientOf(ObjectProperty, TransitiveProperty): pass

    class isToppingOf(isIngredientOf):
        domain = [PizzaTopping]
        range = [Pizza]

    class isBaseOf(isIngredientOf, FunctionalProperty):
        domain = [PizzaBase]
        range = [Pizza]

A continuacion vamos a anadir una restriccion para especificar que cada Pizza debe tener una PizzaBase

In [9]:
with onto:
    Pizza.is_a = [hasBase.some(PizzaBase)]

Podemos crear algunas clases para representar distintos tipos de pizzas. En primer lugar creamos una clase llamada NamedPizza, que es subclase de Pizza.
Crearemos varios tipos de pizza a partir de NamedPizza:

In [10]:
with onto:
    class NamedPizza(Pizza): pass

    class MargheritaPizza(NamedPizza):
        __annotations__ = "A pizza that only has Mozzarella and Tomato toppings"
        hasTopping = [Mozarella, Tomato]

    class AmericanaPizza(NamedPizza):
        hasTopping = [Mozarella, Tomato, Pepperoni]

    class AmericanHotPizza(NamedPizza):
        hasTopping = [Mozarella, Tomato, Pepperoni, JalapenoPepper]

    class SohoPizza(NamedPizza):
        hasTopping = [Mozarella, Tomato, Parmezan, Olive]

    AllDisjoint([MargheritaPizza, AmericanaPizza, AmericanHotPizza, SohoPizza])

Vamos a probar el razonador incluido en owlready2 para ver si detecta inconsistencias en la definicion de clases. Para ello definimos una clase llamada InconsistentTopping, que es subclase de Vegetable y Cheese. De acuerdo con nuestra deficion, ambas clases son disyuntivas por lo que deberia de ser imposible definir una clase perteneciente a ambos ingredientes.

In [11]:
with onto:
    class InconsistentTopping(Vegetable, Cheese):
      __annotations__ = "inconsistency test"

    sync_reasoner_pellet(infer_property_values=True)

    inconsistencies = list(default_world.inconsistent_classes())
    print(f"inconsistencias encontradas: {inconsistencies}")

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /Users/jorge/PycharmProjects/pizzaOwl/venv/lib/python3.9/site-packages/owlready2/pellet/httpclient-4.2.3.jar:/Users/jorge/PycharmProjects/pizzaOwl/venv/lib/python3.9/site-packages/owlready2/pellet/aterm-java-1.6.jar:/Users/jorge/PycharmProjects/pizzaOwl/venv/lib/python3.9/site-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/Users/jorge/PycharmProjects/pizzaOwl/venv/lib/python3.9/site-packages/owlready2/pellet/slf4j-api-1.6.4.jar:/Users/jorge/PycharmProjects/pizzaOwl/venv/lib/python3.9/site-packages/owlready2/pellet/jena-tdb-0.10.0.jar:/Users/jorge/PycharmProjects/pizzaOwl/venv/lib/python3.9/site-packages/owlready2/pellet/jena-iri-0.9.5.jar:/Users/jorge/PycharmProjects/pizzaOwl/venv/lib/python3.9/site-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/Users/jorge/PycharmProjects/pizzaOwl/venv/lib/python3.9/site-packages/owlready2/pellet/log4j-1.2.16.jar:/Users/jorge/PycharmProjects/pizzaOwl/venv/lib/python3.9/site-package

inconsistencias encontradas: [owl.Nothing, pizza.InconsistentTopping]


* Owlready2 * Pellet took 1.945755958557129 seconds
* Owlready * Equivalenting: owl.Nothing pizza.InconsistentTopping
* Owlready * Equivalenting: pizza.InconsistentTopping owl.Nothing
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


El razonador no solo se encarga de encontrar inconsistencias en nuestras definiciones, sino también de clasificar clases sin necesidad de hacerlo por nosotros mismos.
Como demostracion, definimos una subclase de Pizza llamada CheesyPizza, que contiene queso como único ingrediente.

In [12]:
with onto:
    class CheesyPizza(Pizza):
      defined_class = True
      hasTopping = [Cheese]

Antes de ejecutar el razonador, la situacion es:

In [14]:
print('subclases de Pizza', list(Pizza.subclasses()))
print('subclases de CheesyPizza:', list(CheesyPizza.subclasses()))
print('subclases de NamedPizza:', list(NamedPizza.subclasses()))

subclases de Pizza [pizza.NamedPizza, pizza.CheesyPizza]
subclases de CheesyPizza: []
subclases de NamedPizza: [pizza.MargheritaPizza, pizza.AmericanaPizza, pizza.AmericanHotPizza, pizza.SohoPizza]


Ejecutamos el razonador...

In [15]:
sync_reasoner_pellet(infer_property_values=True)

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /Users/jorge/PycharmProjects/pizzaOwl/venv/lib/python3.9/site-packages/owlready2/pellet/httpclient-4.2.3.jar:/Users/jorge/PycharmProjects/pizzaOwl/venv/lib/python3.9/site-packages/owlready2/pellet/aterm-java-1.6.jar:/Users/jorge/PycharmProjects/pizzaOwl/venv/lib/python3.9/site-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/Users/jorge/PycharmProjects/pizzaOwl/venv/lib/python3.9/site-packages/owlready2/pellet/slf4j-api-1.6.4.jar:/Users/jorge/PycharmProjects/pizzaOwl/venv/lib/python3.9/site-packages/owlready2/pellet/jena-tdb-0.10.0.jar:/Users/jorge/PycharmProjects/pizzaOwl/venv/lib/python3.9/site-packages/owlready2/pellet/jena-iri-0.9.5.jar:/Users/jorge/PycharmProjects/pizzaOwl/venv/lib/python3.9/site-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/Users/jorge/PycharmProjects/pizzaOwl/venv/lib/python3.9/site-packages/owlready2/pellet/log4j-1.2.16.jar:/Users/jorge/PycharmProjects/pizzaOwl/venv/lib/python3.9/site-package

Y ahora tenemos que

In [16]:
print('subclases de Pizza', list(Pizza.subclasses()))
print('subclases de CheesyPizza:', list(CheesyPizza.subclasses()))
print('subclases de NamedPizza:', list(NamedPizza.subclasses()))

subclases de Pizza [pizza.NamedPizza, pizza.CheesyPizza]
subclases de CheesyPizza: [pizza.MargheritaPizza, pizza.AmericanaPizza, pizza.AmericanHotPizza, pizza.SohoPizza]
subclases de NamedPizza: [pizza.MargheritaPizza, pizza.AmericanaPizza, pizza.AmericanHotPizza, pizza.SohoPizza]


Como vemos el razonador se ha encargado de incluir las pizzas que habiamos definido antes como subclases de NamedPizza también como subclases de CheesyPizza.
